In [1]:
import pandas as pd
import numpy as np
import re

In [9]:
#get name on input file
input_folder = 'converted_logs/text_files/'
in_file = input('Input file: ')
in_file = input_folder + in_file
#extract all lines, removing newlines characters and whitespace
with open(in_file, 'r') as file:
    lines = [line.strip() for line in file.readlines()]

#all new entries start with a time in format hh:mm
time_pattern = "^([0-9]{2}:[0-9]{2})"
consolidated_entries = []
entry = ""
length_lines = len(lines)
i = 0

#loop through lines, appending all lines regarding one entry together
#into a single string
while i < length_lines:
    #if line starts with a time, start a new entry
    if re.match(time_pattern, lines[i]):        
        entry = lines[i]
        i+=1
        while  i < length_lines and not re.match(time_pattern, lines[i]):
            entry += lines[i]
            i += 1            
    else:
        print('Error in input: non-standard line detected')
        i += 1
    #add complete entry to list
    consolidated_entries.append(entry)

FileNotFoundError: [Errno 2] No such file or directory: 'converted_logs/Press-log-Sep-22nd-thru-28th-2021.txt'

In [7]:
#define regex patterns
location_pattern = "\s(on|at)\s"
id_pattern = "[0-9]{10}"

#give name of write file, or use default nameing scheme of the day of the first entry
out_file = input("Enter name of output file, or type 'default' for default naming scheme(M_D_Y_entries.txt)")
if out_file.lower() == 'default':
    #pull the id from the first entry, which contains date information
    idx = re.search(id_pattern, consolidated_entries[0]).start()
    datestring = consolidated_entries[0][idx:idx+10]
 
    out_file = datestring[2:4]+'_'+datestring[4:6]+'_'+datestring[0:2]

output_folder = 'converted_logs/csvs/'
out_file=out_file+'.csv'

#Write file headers
with open(out_file, 'w') as file:
    file.write("ID,time,category,location,sublocation,description,disposition\n")

#Seperate each entry into field
for entry in consolidated_entries:
    #extract the time (given in military time, always the first 5 characters)
    time = entry[0:5]
    entry = entry[5:]
    
    #find the index of the 10 digit entry id
    id_idx = re.search(id_pattern, entry).start()
    #the category of entry is everything up until the id starts. Quotes
    #are placed around the string to prevent issues in csv conversion.
    category = '\"'+entry[:id_idx].strip()+'\"'
    
    #extract id #
    entry_id = entry[id_idx:id_idx + 10]
    #<TO DO>
    #extract date information from id
    
    #All locations follow a pattern of <various string> [on|at] location, so
    #the first match of on|at is followed by the location name
    occurred_idx = re.search(location_pattern, entry).end()
    entry = entry[occurred_idx:]
    
    #the description will follow the '.' ending the location
    description_idx = entry.find('.')+1
    
    #everything up to the period is grabbed, and unneeded information like town
    #name (all entries are in Fairfax) or common puncutation quirks are removed
    #or standardized. Remaining location should be a street.
    location = entry[0:description_idx].replace('Fairfax', '').replace(', .', '').replace(' ()', '').replace('.', '').replace(', ', '-').strip()
    
    #If the word 'on' remains in the location, that means an area within the 
    #main location is specified. Otherwise, N/A is used as a filler
    if location.find(' on ') > 1:
        idx = location.find(' on ')
        sub_location = '\"'+location[0:idx]+'\"'
        location = location[idx+4:]
    else:
        sub_location = 'N/A'
    location = '\"'+location+'\"'
    entry = entry[description_idx:]
    
    #The entry description is everything up until the word 'Disposition'. Some entries
    #do not have additional desciptive text.
    disposition_idx = entry.find('Disposition:')
    
    #Formatting quirks removed.
    description = '\"'+entry[:disposition_idx].lstrip(". ").replace(" .", '').replace('..', '.').strip()+'\"'
    if description == '':
        description = 'NA'
    
    #Fomatting quirks removed, and a common typo fixed.
    disposition = '\"'+entry[disposition_idx:].replace('Disposition:', '').strip().replace('ServiceProvided.', 'Service Provided.')+'\"'

    #entries saved to file specified by outfile
    with open(out_file,'a') as file:
        file.write(entry_id + ',' + time+ ',' +category+ ',' +location+ ',' + sub_location + ',' +description+ ',' +disposition+'\n')
        